In [2]:
%%bash

pip install --upgrade pip

pip3 install llama-index
pip3 install llama-index-embeddings-huggingface
pip3 install llama-index-retrievers-bm25

Defaulting to user installation because normal site-packages is not writeable
  Using cached pip-24.0-py3-none-any.whl.metadata (3.6 kB)
Using cached pip-24.0-py3-none-any.whl (2.1 MB)
  Attempting uninstall: pip
    Found existing installation: pip 23.3.2
    Uninstalling pip-23.3.2:
      Successfully uninstalled pip-23.3.2
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [1]:

from typing import Any

from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.core import StorageContext, load_index_from_storage
from llama_index.core.llms import (
    CustomLLM,
    CompletionResponse,
    CompletionResponseGen,
    LLMMetadata,
)
# from llama_index.core.llms.base import llm_completion_callback
from llama_index.core import Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.retrievers.bm25 import BM25Retriever
from llama_index.core.node_parser import SentenceSplitter

from llama_index.core.retrievers import VectorIndexRetriever


/home/addo/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The code in the cell below is used to define a custom LLM, such as one queried over a private API server.

In this demo we only look at the retriever.

In [2]:

class OurLLM(CustomLLM):
    context_window: int = 3900
    num_output: int = 256
    model_name: str = "llama2-13b-chat"
    dummy_response: str = "Could not talk to LLM"

    @property
    def metadata(self) -> LLMMetadata:
        """Get LLM metadata."""
        return LLMMetadata(
            context_window=self.context_window,
            num_output=self.num_output,
            model_name=self.model_name,
        )

    # @llm_completion_callback()
    def complete(
            self, prompt: str, formatted: bool = False, **kwargs: Any
    ) -> CompletionResponse:
        """Predict the answer to a query.
        Args:
            prompt (str): Prompt to use for prediction.
        Returns:
            Tuple[str, str]: Tuple of the predicted answer and the formatted prompt.
        """
        result, candidates = self.generate_text(
            model=self.model_name,
            prompt=prompt,
            **kwargs,
        )
        return CompletionResponse(text=result, raw=candidates)

    # @llm_completion_callback()
    def stream_complete(
            self, prompt: str, formatted: bool = False, **kwargs: Any
    ) -> CompletionResponseGen:
        raise NotImplementedError(
            "This module does not support streaming completion currently."
        )

    def generate_text(self, model, prompt, **kwargs):
        return "dummy", "text"

In [3]:
Settings.llm = OurLLM()


# model = "sentence-transformers/all-MiniLM-L6-v2"
# model = deepset/roberta-base-squad2 # gives error
# model = "google-bert/bert-base-uncased"
# Settings.embed_model = HuggingFaceEmbedding(model)
Settings.embed_model = HuggingFaceEmbedding()

In [5]:
# Load embeddings
persist_dir = ".llamaindex_storage"
try:
    storage_context = StorageContext.from_defaults(persist_dir=persist_dir)
    index = load_index_from_storage(storage_context)
except:
    text_splitter = SentenceSplitter(chunk_size=512, chunk_overlap=40)
    documents = SimpleDirectoryReader(
        "data/wiki_gameofthrones_txt1/").load_data()
    index = VectorStoreIndex.from_documents(documents, transformations=[text_splitter])
    index.storage_context.persist(persist_dir=persist_dir)

In [6]:
bm25retriever = BM25Retriever.from_defaults(index=index, similarity_top_k=5)

nodes = bm25retriever.retrieve("Who is the father of Arya Stark?")
print("------------  BM25 retriever ------------------")
for node in nodes:
    print(node)


------------  BM25 retriever ------------------
Node ID: e52a600e-d62e-4cd1-8595-d1bb0dd2dc14
Text: ==In other media==  In his article "The War in Westeros and Just
War Theory", Richard H. Corrigan uses Robb to illustrate the concepts
of just cause and right intention in the decision to go to war: "Robb
is not only fighting this war to ensure that his fellow Northerners
have a just king Robb's cause.  He is also doing it to avenge his
father, ...
Score:  4.300

Node ID: b3f85bb0-091f-4343-ba6d-c3030b065826
Text: In Season 6, he is outraged when he hears of the Blackfish
recapture' of Riverrun and blames his sons Lothar and Black Walder for
allowing him to escape. He then orders them to retake the castle using
Edmure Tully as a hostage. Though they successfully retake Riverrun
with the help of a Lannister host led by Jaime Lannister, Walder is
ambushed s...
Score:  3.757

Node ID: 48da9fcb-08d6-43bb-82cb-37b04b463110
Text: === Season 2 (2012) ===   Ned's oldest son Robb has declared war

In [7]:
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=5,
)


print("------------ Vector retriever ------------------")
nodes = retriever.retrieve("Who is the father of Arya Stark?")
for node in nodes:
    print(node)

------------ Vector retriever ------------------
Node ID: b08d639e-f411-410c-9808-4868bad40bf0
Text: '''Arya Stark''' is a fictional character in American author
George R. R. Martin's ''A Song of Ice and Fire'' epic fantasy novel
series.  She is a prominent point of view character in the novels with
the third most viewpoint chapters, and is the only viewpoint character
to have appeared in every published book of the series.  Introduced in
1996'...
Score:  0.897

Node ID: 9966e5ed-802d-4ce2-86de-31721e92f066
Text: The Stark sisters later resolve their differences, and
acknowledge that the Starks must stay together to survive winter.
====Season 8==== Arya reunites with Jon, Gendry, and the Hound, who
have all journeyed to Winterfell with Daenerys Targaryen's forces to
make a stand against the approaching White Walkers. Arya asks Gendry,
who is forging dra...
Score:  0.887

Node ID: dbeb1f87-620e-42c8-a6cc-21ae8e07221d
Text: She seduces him and lures him to her own room, before slicing
hi